In [1]:
# Install the necessary libraries and clone the repository
!pip install ninja
!pip install torch torchvision
!pip install pillow
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
# Moving to the stylegan2-ada-pytorch directory
import os
os.chdir('stylegan2-ada-pytorch')

In [3]:
# Create a models folder and download the pretreated model
!mkdir models
!wget -O models/ffhq.pkl https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

--2024-06-03 05:37:17--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 52.84.18.79, 52.84.18.33, 52.84.18.74, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|52.84.18.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646055 (364M) [binary/octet-stream]
Saving to: ‘models/ffhq.pkl’

models/ffhq.pkl     100%[===================>] 363.97M   210MB/s    in 1.7s    

2024-06-03 05:37:20 (210 MB/s) - ‘models/ffhq.pkl’ saved [381646055/381646055]



In [4]:
# Checking the current directory. All generated images will be saved in this location.
!pwd

/content/stylegan2-ada-pytorch


In [5]:
# Generating images of ten people
import torch
import pickle
import dnnlib
import legacy
import PIL.Image
import numpy as np

def load_generator(model_path):
    # Determine the device to use (GPU if available, else CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the pretrained model from the given path
    with open(model_path, 'rb') as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)
    return G

def generate_image(generator, seed, truncation_psi=0.5):

    # Determine the device to use (GPU if available, else CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

   # Ensuring repeatability of results
    torch.manual_seed(seed)
    rnd = np.random.RandomState(seed)

    # Generate a random latent vector
    z = torch.tensor(rnd.randn(1, generator.z_dim), device=device)

    # Generate an image using the generator model
    img = generator(z, None, truncation_psi=truncation_psi, noise_mode='const')

    # Convert the generated image to a format suitable for saving
    img = img[0].permute(1, 2, 0).cpu().numpy()
    img = (img * 127.5 + 128).clip(0, 255).astype(np.uint8)

    # Return the image as a PIL Image object
    return PIL.Image.fromarray(img, 'RGB')

# Path to the pretrained model
model_path = 'models/ffhq.pkl'

# Load the generator model
generator = load_generator(model_path)

# List of seeds to generate different images
seeds = [109, 10, 70, 30, 35, 38, 40, 100, 116, 115]

# Generate and save images for each seed
for seed in seeds:

    image = generate_image(generator, seed)
    image.save(f'generated_face_{seed}.png')

Setting up PyTorch plugin "bias_act_plugin"... 

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.


In [7]:
# Add watermarks informing that images are generated by AI.
# It is necessary to add the Arial.ttf file posted in the repository to the /content/stylegan2-ada-pytorch directory.
# Click the folder icon on the left and add the file to the folder named stylegan2-ada-pytorch.
from PIL import Image, ImageDraw, ImageFont
import math

def add_watermark(image_path):
    # Load the image
    image = Image.open(image_path)

    # Set the watermark text and its style
    watermark_text = "generated by AI"
    font_size = 120
    font = ImageFont.truetype("Arial.ttf", font_size)

    # Calculate the diagonal length of the image
    diagonal_length = math.sqrt(image.width ** 2 + image.height ** 2)

    # Calculate the width and height of the text
    text_width, text_height = font.getsize(watermark_text)

    # Calculate the new font size based on the diagonal length of the image
    new_font_size = int(font_size * diagonal_length / (image.width + image.height))
    font = ImageFont.truetype("Arial.ttf", new_font_size)

    # Calculate the position of the text along the diagonal of the image
    x_offset = (image.width - text_width) // 2
    y_offset = (image.height - text_height) // 2

    # Create a new layer with transparency
    watermark = Image.new("RGBA", (image.width, image.height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(watermark)

    # Draw the text on the layer and rotate it by 45 degrees
    draw.text((x_offset, y_offset), watermark_text, font=font, fill=(255, 255, 255, 128))
    watermark_rotated = watermark.rotate(45, expand=True)

    # Resize the watermark image to fit the main image
    watermark_resized = watermark_rotated.resize(image.size, Image.ANTIALIAS)

    # Composite the watermark layer over the original image
    watermarked_image = Image.alpha_composite(image.convert("RGBA"), watermark_resized)

    # Save the image with the watermark
    watermarked_image.save("watermarked_" + image_path)

# Adding a watermark to all previously generated images. Images with watermarks are also saved in current directory.

seeds = [109, 10, 70, 30, 35, 38, 40, 100, 116, 115]
for seed in seeds:
    add_watermark(f'generated_face_{seed}.png')

<ipython-input-7-0f6977195840>:20: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width, text_height = font.getsize(watermark_text)
<ipython-input-7-0f6977195840>:39: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  watermark_resized = watermark_rotated.resize(image.size, Image.ANTIALIAS)
